# Parameters

In [ ]:
SLAKTESTATISTIKK_URL = 'https://animalia.no/no/ravare-og-foredling/klassifisering/klassifisering-av-storfe/'
ANIMALIA_URL = 'https://animalia.no/'

EXCEL_LINK_INDICATOR = '.xlsx'

# data to save
KATEGORI = ['Ung okse']
AAR = ['2024']

In [ ]:
NOTERINGSPRIS_FJØRFE_URL = 'https://www.landbruksdirektoratet.no/nb/industri-og-handel/marked-og-pris/priser-og-prisuttak/noteringspris-fjorfe'

NOTERINGSPRIS_FJØRFE_FILE_LINK_NAME = 'Tallgrunnlag for graf'
NOTERINGSPRIS_COLUMN_NAME = 'noteringspriser_fjorfe'

# Import Libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import json
import copy
import pandas as pd

from google.colab import files
import re
import numpy as np
from google.colab import drive
import os

In [ ]:
drive.mount('drive')

Mounted at drive


# Slaktestatistikk Task

Web scrape slaktedata from Slaktestatistikk






# Home Page

In the Norwegian language text, we have to properly encode the Norwegian alphabet

In [ ]:
page = requests.get(SLAKTESTATISTIKK_URL)
page.encoding = page.apparent_encoding
home_webpage = BeautifulSoup(page.text, 'html')

# Retrieve Art

In [ ]:
title = home_webpage.find('title')

print(title) # Prints the tag
print(title.string) # Prints the tag string content

<title>Slaktestatistikk - storfe</title>
Slaktestatistikk - storfe


In [ ]:
title_text = title.string
art = title_text.split(' - ')[1]
art

'storfe'

# Retrieve Art data from excel files

In [ ]:
year_pattern = r'.*(20[0-9]{2})'
kategori_urls = {}

for a in home_webpage.find_all('a', href = True):

    if EXCEL_LINK_INDICATOR in a['href']:

        kategori_url = ANIMALIA_URL + a['href']
        kategori_navn = a.get_text()

        # Use the following code to remove all '\xa0'
        kategori_navn_list = kategori_navn.split()
        kategori_navn = ' '.join(kategori_navn_list)

        if kategori_navn not in kategori_urls:
            kategori_urls[kategori_navn] = {}

        match = re.match(year_pattern, kategori_url)
        if match is not None:

            aar = match.group(1)
            kategori_urls[kategori_navn][aar] = kategori_url

# kategori_urls

Print all types of excel files available

In [ ]:
# for kategori, kategori_df_per_year in kategori_urls.items():
#     print(f'kategori: {kategori}')
#     for aar, kategori_df in kategori_df_per_year.items():
#         print(f'aar: {aar}')

#     print()

Retrieve excel file from given url

In [ ]:
kategori_files = {}

for kategori, url_per_aar in kategori_urls.items():
    kategori_files[kategori] = {}

    for year, url in url_per_aar.items():
        kategori_file = pd.ExcelFile(url)
        kategori_files[kategori][year] = kategori_file

Read specific interesting to us data from excel files and save it into dataframes

Ønsker at disse gulfargede kolonnene leses inn til tabellen:

Kolonne B år -> til kolonne aar

Kolonne C ukenr -> til kolonne uke

Kolonne F antall slakt -> til kolonne antall_slakt

Kolonne K vekt i kg -> til kolonne vekt_kg

Kolonne P klasse -> til kolonne klasse

Kolonne U fettgruppe mid -> til kolonne fettgruppe

In [ ]:
required_columns = {'År': 'aar', 'Uke': 'uke', 'Antall slakt': 'antall_slakt', 'Vekt': 'vekt_kg', 'Klasse': 'klasse', 'Fettgruppe Mid': 'fettgruppe'}

not_needed_columns_indicators = ['%', '+/-', 'Standardavvikelser', 'Andels', 'Middel', 'Produsenter', 'dager', 'Tilvekst ']

In [ ]:
kategori_dataframes = {}

for kategori, xl_files_per_year in kategori_files.items():

    kategori_dataframes[kategori] = {}
    for year, xl_file in xl_files_per_year.items():

        for skip_rows in range(3, 10):

            df = xl_file.parse('Uke', header = [0, 1, 2, 3], skiprows = skip_rows)
            if isinstance( df.iloc[0, 1], np.float64 ):

                df.columns = [' '.join([word for word in col if 'Unnamed' not in word]) for col in df.columns]
                df.columns = [col.replace('- ', '') for col in df.columns]

                selected_df_columns = [
                    col for col in df.columns if any(
                        substring in col for substring in required_columns
                        )
                    ]
                selected_df_columns = [
                    col for col in df.columns if not any(
                        substring in col for substring in not_needed_columns_indicators
                        )
                    ]
                selected_df_columns = filter(bool, selected_df_columns)
                kategori_dataframes[kategori][year] = df[selected_df_columns]

                break

Print current columns

In [ ]:
for kategori, kategori_df_per_year in kategori_dataframes.items():
    for aar, kategori_df in kategori_df_per_year.items():
        print()
        print(f'kategori: {kategori}, aar: {aar}')
        print(*kategori_df.columns, sep = ', ')


kategori: Alt storfe - overordnet, aar: 2024
År, Ukenr, Antall Antall slakt, Antall Vekt, Antall Klasse, Antall Fettgruppe Mid.

kategori: Alt storfe - overordnet, aar: 2023
År, Ukenr, Antall Antall slakt, Antall Klasse, Antall Fettgruppe Mid., Antall Vekt

kategori: Kalv, aar: 2024
År, Ukenr, Antall Antall slakt, Antall Vekt Vekt, Antall Klasse, Antall Fettgruppe Mid.

kategori: Kalv, aar: 2023
År, Ukenr, Antall Antall slakt, Antall Klasse, Antall KlaVekt, Antall Fettgruppe Mid.

kategori: Ung okse, aar: 2024
År, Ukenr, Antall Antall slakt, Antall Vekt i kg, Antall Klasse, Antall Fettgruppe Mid.

kategori: Ung okse, aar: 2023
År, Ukenr, Antall Antall slakt, Antall Klasse, Antall Vekt i kg, Antall Fettgruppe Mid.

kategori: Okse, aar: 2024
År, Ukenr, Antall Antall slakt, Antall Klasse, Antall Fettgruppe Mid., Antall Vekt

kategori: Okse, aar: 2023
År, Ukenr, Antall Antall slakt, Antall Klasse, Antall Fettgruppe Mid., Antall Vekt

kategori: Kastrat, aar: 2024
År, Ukenr, Antall Antall s

# Transform data

Remove empty rows

In [ ]:
for kategori, kategori_df_per_year in kategori_dataframes.items():
    for aar, kategori_df in kategori_df_per_year.items():

        print(f' kategori: {kategori},  aar: {aar}')
        print(f'shape before removing empty rows: {kategori_df_per_year[aar].shape}')
        kategori_df = kategori_df.replace('', np.nan)
        kategori_df = kategori_df.dropna(thresh = 1)
        kategori_df_per_year[aar] = kategori_df
        print(f'shape after removing empty rows: {kategori_df_per_year[aar].shape}')

 kategori: Alt storfe - overordnet,  aar: 2024
shape before removing empty rows: (106, 6)
shape after removing empty rows: (104, 6)
 kategori: Alt storfe - overordnet,  aar: 2023
shape before removing empty rows: (159, 6)
shape after removing empty rows: (156, 6)
 kategori: Kalv,  aar: 2024
shape before removing empty rows: (106, 6)
shape after removing empty rows: (104, 6)
 kategori: Kalv,  aar: 2023
shape before removing empty rows: (265, 6)
shape after removing empty rows: (260, 6)
 kategori: Ung okse,  aar: 2024
shape before removing empty rows: (106, 6)
shape after removing empty rows: (104, 6)
 kategori: Ung okse,  aar: 2023
shape before removing empty rows: (265, 6)
shape after removing empty rows: (260, 6)
 kategori: Okse,  aar: 2024
shape before removing empty rows: (105, 6)
shape after removing empty rows: (104, 6)
 kategori: Okse,  aar: 2023
shape before removing empty rows: (159, 6)
shape after removing empty rows: (156, 6)
 kategori: Kastrat,  aar: 2024
shape before removi

Rename columns

In [ ]:
required_columns

{'År': 'aar',
 'Uke': 'uke',
 'Antall slakt': 'antall_slakt',
 'Vekt': 'vekt_kg',
 'Klasse': 'klasse',
 'Fettgruppe Mid': 'fettgruppe'}

In [ ]:
for kategori, kategori_df_per_year in kategori_dataframes.items():
    for aar, kategori_df in kategori_df_per_year.items():

        columns = kategori_df.columns

        renamed_columns = {}
        for col in columns:
            for column_indicator in required_columns:
                if column_indicator in col:
                    renamed_columns[col] = required_columns[column_indicator]

        kategori_df = kategori_df.rename(columns = renamed_columns)
        kategori_df['year of file'] = aar
        kategori_df_per_year[aar] = kategori_df

Combine dataframes for different years

In [ ]:
for kategori, kategori_df_per_year in kategori_dataframes.items():

    data_per_year = list(kategori_df_per_year.values())
    kategori_df = pd.concat(data_per_year, ignore_index = True, axis = 0)
    kategori_dataframes[kategori] = kategori_df

Add category and art columns

In [ ]:
for kategori, kategori_df_per_year in kategori_dataframes.items():
    kategori_df_per_year['kategory'] = kategori
    kategori_df_per_year['art'] = art

Combine all dataframes together

In [ ]:
art_dataframes = list(kategori_dataframes.values())
art_dataframe = pd.concat(art_dataframes, ignore_index = True, axis = 0)

In [ ]:
art_dataframe.shape

(2392, 9)

In [ ]:
art_dataframe.head()

,aar,uke,antall_slakt,vekt_kg,klasse,fettgruppe,year of file,kategory,art
0,2024.0,1.0,6062.0,276.515853,4.980350,7.388651,2024,Alt storfe - overordnet,storfe
1,2024.0,2.0,7137.0,277.477974,4.983722,7.298445,2024,Alt storfe - overordnet,storfe
2,2024.0,3.0,6181.0,277.534768,4.988817,7.318557,2024,Alt storfe - overordnet,storfe
3,2024.0,4.0,6147.0,280.522434,5.105195,7.255409,2024,Alt storfe - overordnet,storfe
4,2024.0,5.0,5490.0,270.246284,4.978661,7.048998,2024,Alt storfe - overordnet,storfe


In [ ]:
art_dataframe.shape

(2392, 9)

# Noteringspris fjørfe Task


Web scrape Notert pris from Noteringspris fjørfe

In [ ]:
page = requests.get(NOTERINGSPRIS_FJØRFE_URL)
page.encoding = page.apparent_encoding
home_webpage = BeautifulSoup(page.text, 'html')

In [ ]:
for a in home_webpage.find_all('a', href = True):
    if NOTERINGSPRIS_FJØRFE_FILE_LINK_NAME == a.get_text():

        notert_pris_file_url = a['href']

        notert_pris_file_html = requests.get(notert_pris_file_url).text
        notert_pris_file_soup = BeautifulSoup(notert_pris_file_html, 'lxml')

        tables = notert_pris_file_soup.find_all('table')
        table = tables[0]

        with open('tallgrunnlag.csv', 'w', encoding = 'utf8') as f:
            wr = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC, lineterminator = '\n')
            wr.writerows([[td.text for td in row.find_all("td")] for row in table.find_all("tr")])

            notert_pris_df = pd.read_csv('tallgrunnlag.csv')

In [ ]:
notert_pris_df.head()

,Dato,Referansepris,Øvrepris,Notert pris,Avvik fra ref.pris,Akk. gj.sn. engrospris,Akk. avvik,Ukenr.,Avtaleår,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,17.02.2025,"37,04","40,75","37,99","0,94","37,50","0,46",8,2024-2023,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.02.2025,"37,04","40,75","37,59","0,55","37,49","0,44",7,2024-2024,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03.02.2025,"37,04","40,75","37,38","0,33","37,49","0,44",6,2024-2025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27.01.2025,"37,04","40,75","37,52","0,47","37,49","0,44",5,2024-2025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20.01.2025,"37,04","40,75","37,62","0,58","37,49","0,44",4,2024-2025,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop unnamed columns

In [ ]:
notert_pris_df = notert_pris_df[notert_pris_df.columns.drop(list(notert_pris_df.filter(regex = 'Unnamed:')))]

In [ ]:
notert_pris_df.head()

,Dato,Referansepris,Øvrepris,Notert pris,Avvik fra ref.pris,Akk. gj.sn. engrospris,Akk. avvik,Ukenr.,Avtaleår
0,17.02.2025,"37,04","40,75","37,99","0,94","37,50","0,46",8,2024-2023
1,10.02.2025,"37,04","40,75","37,59","0,55","37,49","0,44",7,2024-2024
2,03.02.2025,"37,04","40,75","37,38","0,33","37,49","0,44",6,2024-2025
3,27.01.2025,"37,04","40,75","37,52","0,47","37,49","0,44",5,2024-2025
4,20.01.2025,"37,04","40,75","37,62","0,58","37,49","0,44",4,2024-2025


# Save Data

Select required data

In [ ]:
KATEGORI

['Ung okse']

In [ ]:
AAR

['2024']

In [ ]:
dataframe_to_save = art_dataframe[art_dataframe['kategory'].isin(KATEGORI)]
dataframe_to_save = dataframe_to_save[dataframe_to_save['year of file'].isin(AAR)]
dataframe_to_save.drop('year of file', axis = 1, inplace = True)
dataframe_to_save.head()

,aar,uke,antall_slakt,vekt_kg,klasse,fettgruppe,kategory,art
624,2024.0,1.0,2201.0,321.396865,6.054140,7.239891,Ung okse,storfe
625,2024.0,2.0,3111.0,314.484121,5.911869,7.111540,Ung okse,storfe
626,2024.0,3.0,2805.0,313.961034,5.799714,7.239572,Ung okse,storfe
627,2024.0,4.0,2832.0,324.779096,6.098552,7.288136,Ung okse,storfe
628,2024.0,5.0,2417.0,312.418825,5.923873,7.071163,Ung okse,storfe


In [ ]:
dataframe_to_save.to_excel('ung_okse_2024_dataframe.xlsx', index = False)

In [ ]:
!cp ung_okse_2024_dataframe.xlsx 'drive/My Drive/'

In [ ]:
notert_pris_df.to_excel('notert_pris_dataframe.xlsx', index = False)

In [ ]:
!cp notert_pris_dataframe.xlsx 'drive/My Drive/'